# Orchestrating Jobs, Model Registration, and Continuous Deployment with Amazon SageMaker

Amazon SageMaker offers Machine Learning application developers and Machine Learning operations engineers the ability to orchestrate SageMaker jobs and author reproducible Machine Learning pipelines, deploy custom-build models for inference in real-time with low latency or offline inferences with Batch Transform, and track lineage of artifacts. You can institute sound operational practices in deploying and monitoring production workflows, deployment of model artifacts, and track artifact lineage through a simple interface, adhering to safety and best-practice paradigmsfor Machine Learning application development.

The SageMaker Pipelines service supports a SageMaker Machine Learning Pipeline Domain Specific Language (DSL), which is a declarative Json specification. This DSL defines a Directed Acyclic Graph (DAG) of pipeline parameters and SageMaker job steps. The SageMaker Python Software Developer Kit (SDK) streamlines the generation of the pipeline DSL using constructs that are already familiar to engineers and scientists alike.

The SageMaker Model Registry is where trained models are stored, versioned, and managed. Data Scientists and Machine Learning Engineers can compare model versions, approve models for deployment, and deploy models from different AWS accounts, all from a single Model Registry. SageMaker enables customers to follow the best practices with ML Ops and getting started right. Customers are able to standup a full ML Ops end-to-end system with a single API call.

## SageMaker Pipelines

Amazon SageMaker Pipelines support the following activites:

* Pipelines - A Directed Acyclic Graph of steps and conditions to orchestrate SageMaker jobs and resource creation.
* Processing Job steps - A simplified, managed experience on SageMaker to run data processing workloads, such as feature engineering, data validation, model evaluation, and model interpretation.
* Training Job steps - An iterative process that teaches a model to make predictions by presenting examples from a training dataset.
* Conditional step execution - Provides conditional execution of branches in a pipeline.
* Registering Models - Creates a model package resource in the Model Registry that can be used to create deployable models in Amazon SageMaker.
* Creating Model steps - Create a model for use in transform steps or later publication as an endpoint.
* Parameterized Pipeline executions - Allows pipeline executions to vary by supplied parameters.
* Transform Job steps - A batch transform to preprocess datasets to remove noise or bias that interferes with training or inference from your dataset, get inferences from large datasets, and run inference when you don't need a persistent endpoint.

## Layout of the CLOUD-D RF SageMaker Project

```
|-- code
|   |-- cloudd_rf
|   |   |-- datagen
|   |   |   |-- __init__.py
|   |   |   |-- burst_def.py
|   |   |   |-- filters.py
|   |   |   |-- imagedata_gen.py
|   |   |   |-- iqdata_gen.py
|   |   |   |-- modems.py
|   |-- baseline_fusion.py
|   |-- evaluation.py
|   |-- feature_contribution.py
|   |-- feature_extraction.py
|   |-- plot_accuracy_vs_nuisance_params.py
|   |-- preprocessing.py
|   |-- requirements.txt
|   |-- rl_rfe_fusion.py
|   |-- baseline_fusion.py
|   |-- team1_model.py
|   |-- team2_model.py
|   |-- team3_model.py
|   |-- team4_model.py
|   |-- train.py
|-- pipelines
|   |-- cloudd_rf
|   |   |-- __init__.py
|   |   |-- pipeline.py
|   |-- get_pipeline_definition.py
|   |-- __init__.py
|   |-- run_pipeline.py
|   |-- _utils.py
|   `-- __version__.py
|-- scripts
|   |-- baseline_fusion.ipynb
|   |-- data_preprocessing.ipynb
|   |-- feature_contribution.ipynb
|   |-- feature_extraction.ipynb
|   |-- model_evaluation.ipynb
|   |-- plot_accuracy_vs_nuisance_params.ipynb
|   |-- rl_rfe_fusion.ipynb
|   |-- train_team_models.ipynb
|-- README.md
|-- sagemaker-pipelines-project.ipynb
```

A description of some of the artifacts is provided below:
<br/><br/>
Your pipeline artifacts, which includes a pipeline module defining the required `get_pipeline` method that returns an instance of a SageMaker pipeline, a preprocessing script that is used in feature engineering, and a model evaluation script to measure the Mean Squared Error of the model that's trained by the pipeline:

```
|-- pipelines
|   |-- cloudd_rf
|   |   |-- __init__.py
|   |   |-- pipeline.py
```

<br/><br/>
Utility modules for getting pipeline definition jsons and running pipelines:

```
|-- pipelines
|   |-- get_pipeline_definition.py
|   |-- __init__.py
|   |-- run_pipeline.py
|   |-- _utils.py
|   `-- __version__.py
```

### A SageMaker Pipeline

The pipeline that we create follows a typical Machine Learning Application pattern of pre-processing, training, evaluation, and conditional model registration and publication, if the quality of the model is sufficient.

In [2]:
import boto3
import sagemaker


region = boto3.Session().region_name
role = sagemaker.get_execution_role()
default_bucket = sagemaker.session.Session().default_bucket()

# Change these to reflect your project/business name or if you want to separate ModelPackageGroup/Pipeline from the rest of your team
model_package_group_name = f"ClouddRFModels"
pipeline_name = f"Cloud-D-RF-Pipeline"

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


### Get the pipeline instance

Here we get the pipeline instance from your pipeline module so that we can work with it.

In [3]:
from pipelines.cloudd_rf.pipeline import get_pipeline


pipeline = get_pipeline(
    region=region,
    role=role,
    default_bucket=default_bucket,
    model_package_group_name=model_package_group_name,
    pipeline_name=pipeline_name,
)

INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
/opt/conda/lib/python3.10/site-packages/sagemaker/workflow/pipeline_context.py:332: UserWarning: Running within a PipelineSession, there will be No Wait, No Logs, and No Job being started.
  warnings.warn(
INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.


### Submit the pipeline to SageMaker and start execution

Let's submit our pipeline definition to the workflow service. The role passed in will be used by the workflow service to create all the jobs defined in the steps.

In [4]:
pipeline.upsert(role_arn=role)

INFO:sagemaker.processing:Uploaded code to s3://sagemaker-us-east-1-956631329550/Cloud-D-RF-Pipeline/code/25544e07d7a86bbb94493ac8fd53ccf6/sourcedir.tar.gz
INFO:sagemaker.processing:runproc.sh uploaded to s3://sagemaker-us-east-1-956631329550/Cloud-D-RF-Pipeline/code/54f0ef6bee583ff9186b762aaf572190/runproc.sh
INFO:sagemaker.processing:Uploaded code to s3://sagemaker-us-east-1-956631329550/cloudd-rf/analysis/feature_contribution/code/Cloud-D-RF-Pipeline/code/5abc061a9afb5dc63859065f0e7d746b/sourcedir.tar.gz
INFO:sagemaker.processing:runproc.sh uploaded to s3://sagemaker-us-east-1-956631329550/Cloud-D-RF-Pipeline/code/ed478cdc6961095589d2d188a167aaa4/runproc.sh
INFO:sagemaker.processing:Uploaded code to s3://sagemaker-us-east-1-956631329550/cloudd-rf/analysis/feature_extraction/code/Cloud-D-RF-Pipeline/code/9337bc08efd01495c4877747454b09a6/sourcedir.tar.gz
INFO:sagemaker.processing:runproc.sh uploaded to s3://sagemaker-us-east-1-956631329550/Cloud-D-RF-Pipeline/code/19928e857806a8f854a2

{'PipelineArn': 'arn:aws:sagemaker:us-east-1:956631329550:pipeline/Cloud-D-RF-Pipeline',
 'ResponseMetadata': {'RequestId': 'a566a50e-e75e-430c-99c0-a1d92f51b4fd',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'a566a50e-e75e-430c-99c0-a1d92f51b4fd',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '87',
   'date': 'Fri, 01 Nov 2024 20:05:14 GMT'},
  'RetryAttempts': 0}}

We'll start the pipeline, accepting all the default parameters.

Values can also be passed into these pipeline parameters on starting of the pipeline, and will be covered later. 

In [5]:
execution = pipeline.start(
    parameters=dict(
        ModelApprovalStatus="Approved",
        ProcessingInstanceCount=1,
        ProcessingInstanceType="ml.c5.18xlarge",
        FusionInstanceType="ml.g5.xlarge",
        TrainingInstanceCount=1,
        TrainingInstanceType="ml.g5.xlarge",
        InferenceInstanceType="ml.m5.large",
        ChunkSize=100,
        SampleSize=1000,
        NumSensors=4,
        BatchSize=1
    )
)

### Pipeline Operations: examining and waiting for pipeline execution

Now we describe execution instance and list the steps in the execution to find out more about the execution.

In [7]:
execution.describe()

{'PipelineArn': 'arn:aws:sagemaker:us-east-1:956631329550:pipeline/Cloud-D-RF-Pipeline',
 'PipelineExecutionArn': 'arn:aws:sagemaker:us-east-1:956631329550:pipeline/Cloud-D-RF-Pipeline/execution/17jvrgky5kgf',
 'PipelineExecutionDisplayName': 'execution-1730307694386',
 'PipelineExecutionStatus': 'Executing',
 'PipelineExperimentConfig': {'ExperimentName': 'cloud-d-rf-pipeline',
  'TrialName': '17jvrgky5kgf'},
 'CreationTime': datetime.datetime(2024, 10, 30, 17, 1, 34, 328000, tzinfo=tzlocal()),
 'LastModifiedTime': datetime.datetime(2024, 10, 30, 17, 1, 34, 328000, tzinfo=tzlocal()),
 'CreatedBy': {'UserProfileArn': 'arn:aws:sagemaker:us-east-1:956631329550:user-profile/d-at9gbuwoeuur/dev-user',
  'UserProfileName': 'dev-user',
  'DomainId': 'd-at9gbuwoeuur',
  'IamIdentity': {'Arn': 'arn:aws:sts::956631329550:assumed-role/AmazonSageMaker-ExecutionRole-20220708T111991/SageMaker',
   'PrincipalId': 'AROA55O5LVMHPEJLJGI34:SageMaker'}},
 'LastModifiedBy': {'UserProfileArn': 'arn:aws:sage

We can list the execution steps to check out the status and artifacts:

In [8]:
execution.list_steps()

[{'StepName': 'PreprocessData',
  'StartTime': datetime.datetime(2024, 10, 30, 17, 1, 35, 616000, tzinfo=tzlocal()),
  'StepStatus': 'Executing',
  'Metadata': {'ProcessingJob': {'Arn': 'arn:aws:sagemaker:us-east-1:956631329550:processing-job/pipelines-17jvrgky5kgf-PreprocessData-qXElGyYTff'}},
  'AttemptCount': 1}]